### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Data

In [247]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_row', None)



In [248]:
master_user = pd.read_csv('365-report/users_4_7_2022 2 48 41 AM.csv')
master_user.columns = map(str.lower, master_user.columns)


In [249]:
master_user.head()

,alternate email addresses,block credential,city,country/region,department,dirsyncenabled,display name,fax,first name,last dirsync time,last name,last password change time stamp,license assignment details,licenses,mobile phone,oath token meta data,object id,office,password never expires,phone number,postal code,preferred data location,preferred language,proxy addresses,release track,soft deletion time stamp,state,street address,strong password required,title,usage location,user principal name,when created
0,NaN,False,JR.KANTOR PERWAKILAN BI JEMBER,NaN,JR.KANTOR PERWAKILAN BI JEMBER,True,A Robith Mubarok,NaN,A,2022-03-27 23:55:49Z,ROBITH MUBAROK,2022-03-27 23:38:01Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,479496fa-3a1b-445c-8ab5-776137c74151,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,NaN,NaN,NaN,NaN,NaN,smtp:ar_mubarok@bankindonesiagov.mail.onmicros...,NaN,NaN,KPWDN,JR.KANTOR PERWAKILAN BI JEMBER \ - \ -,NaN,Pelaksana,ID,ar_mubarok@bi.go.id,2019-07-18 07:57:32Z
1,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,True,A. Amirullah Amir,NaN,A.,2022-03-27 23:25:44Z,AMIRULLAH AMIR,2022-03-27 23:20:10Z,NaN,Office 365 E3+Microsoft Power Automate Free+En...,NaN,NaN,0c7cb274-878f-46b6-9f19-4edb10e0df14,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,NaN,3017519,NaN,NaN,NaN,smtp:amirullah@bankindonesiagov.mail.onmicroso...,NaN,NaN,KP,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH \ -...,NaN,Manajer,ID,amirullah@bi.go.id,2019-07-18 07:55:50Z
2,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,True,A. Donanto H. W.,NaN,ALOYSIUS,2022-03-25 10:54:17Z,DONANTO H.W.,2022-03-22 23:41:21Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,059936c1-d6d9-433b-a959-cee77e73b530,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,NaN,3018840,NaN,NaN,NaN,smtp:donanto@bankindonesiagov.mail.onmicrosoft...,NaN,NaN,KP,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYA...,NaN,Direktur,ID,donanto@bi.go.id,2019-07-18 07:55:29Z
3,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,BINS.INSTITUT BI,True,A. Farid Aulia,NaN,A.,2022-03-30 05:57:42Z,FARID AULIA,2022-03-30 05:36:21Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,ea16f839-58d8-4a10-b4dd-7ab017acb051,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,NaN,3017812,NaN,NaN,NaN,smtp:arasta@bankindonesiagov.mail.onmicrosoft....,NaN,NaN,KP,BINS.INSTITUT BI \ BINS.GRUP PENGEMBANGAN AKAD...,NaN,Deputi Direktur,ID,arasta@bi.go.id,2019-07-18 07:54:50Z
4,NaN,False,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,NaN,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,True,A. Fauzi Batubara,NaN,ACHMAD,2022-03-23 00:52:45Z,FAUZI BATUBARA,2022-03-23 00:30:35Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,a08e63dd-c294-4692-8ea0-dce4db4ffc3f,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,NaN,3278221,NaN,NaN,NaN,smtp:afauzi_b@bankindonesiagov.mail.onmicrosof...,NaN,NaN,KPWDN,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTA...,NaN,Manajer,ID,afauzi_b@bi.go.id,2019-07-18 07:59:29Z


In [250]:
master_user['department'].value_counts()

DSDM.DEPARTEMEN SUMBER DAYA MANUSIA                             435
DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI                    264
DPU.DEPARTEMEN PENGELOLAAN UANG                                 251
DPLF.DEPARTEMEN PENGELOLAAN LOGISTIK & FASILITAS                159
DPPT.DEPARTEMEN JASA PERBANKAN PERIZINAN&OPERASIONAL TRESURI    145
DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN                 145
DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN               137
DMST.DEPARTEMEN MANAJEMEN STRATEGIS & TATA KELOLA               135
DSTA.DEPARTEMEN STATISTIK                                       131
SB.KANTOR PERWAKILAN BI PROVINSI JAWA TIMUR                     113
DSSK.DEPARTEMEN SURVEILANS SISTEM KEUANGAN                      105
BINS.INSTITUT BI                                                105
DKEU.DEPARTEMEN KEUANGAN                                        104
SM.KANTOR PERWAKILAN BI PROVINSI JAWA TENGAH                    104
DKSP.DEPARTEMEN KEBIJAKAN SISTEM PEMBAYARAN     

In [251]:
user_activity_detail_onedrive = pd.read_csv('365-report/user-detail/OneDriveActivityUserDetail4_7_2022 2 36 05 PM.csv')
user_activity_detail_onedrive.columns = map(str.lower, user_activity_detail_onedrive.columns)


In [252]:
user_activity_detail_onedrive.head()

,report refresh date,user principal name,is deleted,deleted date,last activity date,viewed or edited file count,synced file count,shared internally file count,shared externally file count,assigned products,report period
0,2022-04-05,uma_ak.i@bi.go.id,False,NaN,2022-04-05,204,2768,80,0,MICROSOFT TEAMS EXPLORATORY,180
1,2022-04-05,sriyulia@bi.go.id,False,NaN,2022-02-23,4,0,0,0,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3,180
2,2022-04-05,andree@bi.go.id,False,NaN,2021-12-08,1,0,0,0,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,180
3,2022-04-05,dwi_as@bi.go.id,False,NaN,2022-04-01,353,0,8,0,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,180
4,2022-04-05,indra_m@bi.go.id,False,NaN,2022-03-24,19,0,0,0,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,180


In [265]:
user_activity_detail_sharepoint = pd.read_csv('365-report/user-detail/SharePointActivityUserDetail4_7_2022 2 35 17 PM.csv')
user_activity_detail_sharepoint.columns = map(str.lower, user_activity_detail_sharepoint.columns)
user_activity_detail_sharepoint.head()

,report refresh date,user principal name,is deleted,deleted date,last activity date,viewed or edited file count,synced file count,shared internally file count,shared externally file count,visited page count,assigned products,report period
0,2022-04-05,uma_ak.i@bi.go.id,False,NaN,2022-04-05,2,0,0,0,5,MICROSOFT TEAMS EXPLORATORY,180
1,2022-04-05,sriyulia@bi.go.id,False,NaN,2021-12-22,2,0,0,0,7,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3,180
2,2022-04-05,andree@bi.go.id,False,NaN,2021-10-07,0,0,0,0,0,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,180
3,2022-04-05,dwi_as@bi.go.id,False,NaN,2022-03-25,73,0,0,0,23,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,180
4,2022-04-05,indra_m@bi.go.id,False,NaN,2022-03-01,30,0,0,0,9,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,180


In [266]:
user_sharepoint = user_activity_detail_sharepoint[['user principal name']]
user_sharepoint['last activity date sharepoint'] = user_activity_detail_sharepoint['last activity date']
user_sharepoint.head()

/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user principal name,last activity date sharepoint
0,uma_ak.i@bi.go.id,2022-04-05
1,sriyulia@bi.go.id,2021-12-22
2,andree@bi.go.id,2021-10-07
3,dwi_as@bi.go.id,2022-03-25
4,indra_m@bi.go.id,2022-03-01


In [256]:
user_onedrive = user_activity_detail_onedrive[['user principal name']]
user_onedrive['last activity date onedrive'] = user_activity_detail_onedrive['last activity date']
user_onedrive['report refresh date'] = user_activity_detail_onedrive['report refresh date']
user_onedrive.head()

/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,user principal name,last activity date onedrive,report refresh date
0,uma_ak.i@bi.go.id,2022-04-05,2022-04-05
1,sriyulia@bi.go.id,2022-02-23,2022-04-05
2,andree@bi.go.id,2021-12-08,2022-04-05
3,dwi_as@bi.go.id,2022-04-01,2022-04-05
4,indra_m@bi.go.id,2022-03-24,2022-04-05


In [264]:
user_activity_detail_teams = pd.read_csv('365-report/user-detail/TeamsUserActivityUserDetail4_7_2022 2 34 43 PM.csv')
user_activity_detail_teams.columns = map(str.lower, user_activity_detail_teams.columns)
user_activity_detail_teams.head()

,report refresh date,user id,user principal name,last activity date,is deleted,deleted date,assigned products,team chat message count,private chat message count,call count,meeting count,meetings organized count,meetings attended count,ad hoc meetings organized count,ad hoc meetings attended count,scheduled one-time meetings organized count,scheduled one-time meetings attended count,scheduled recurring meetings organized count,scheduled recurring meetings attended count,audio duration,video duration,screen share duration,audio duration in seconds,video duration in seconds,screen share duration in seconds,has other action,urgent messages,post messages,reply messages,is licensed,report period
0,2022-04-05,650ad1d9-c6c3-4f58-9398-abce5ca24566,uma_ak.i@bi.go.id,2022-04-05,False,NaN,MICROSOFT TEAMS EXPLORATORY,0,479,38,38,0,38,0,10,8,26,0,0,P3DT32M22S,P1DT20H5M39S,P2DT16H50M53S,261142,158739,233453,Yes,0,0,0,Yes,180
1,2022-04-05,7d15390c-e9c1-4b92-8dee-e5ae5df27b8b,sriyulia@bi.go.id,2022-04-05,False,NaN,OFFICE 365 E3+ENTERPRISE MOBILITY + SECURITY E3,0,9,27,179,0,179,5,105,0,74,0,0,P10DT21H21M2S,P10DT2H21M22S,P9DT12H18M31S,940862,872482,821911,Yes,0,0,0,Yes,180
2,2022-04-05,c6cca137-709d-4266-b9ed-d2736c49d462,andree@bi.go.id,2022-01-26,False,NaN,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,0,2,1,0,0,0,0,0,0,0,0,0,PT27M2S,PT27M2S,PT27M2S,1622,1622,1622,Yes,0,0,0,Yes,180
3,2022-04-05,c0ef6f8f-199b-47ba-bf4d-f85868f8a5e4,dwi_as@bi.go.id,2022-04-05,False,NaN,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E...,0,783,33,49,0,49,1,5,42,28,0,0,P3DT19H,P3DT4H54M2S,P3DT18H14M33S,327600,276842,324873,Yes,0,0,0,Yes,180
4,2022-04-05,4e3a62a6-c0a2-471b-9e61-6e08327117ad,indra_m@bi.go.id,2022-04-04,False,NaN,ENTERPRISE MOBILITY + SECURITY E3+OFFICE 365 E3,0,0,0,8,0,8,0,0,0,8,0,0,PT17H11M45S,PT17H11M45S,PT12H20M3S,61905,61905,44403,Yes,0,0,0,Yes,180


In [263]:
user_teams = user_activity_detail_teams[['user principal name']]
user_teams['last activity date teams'] = user_activity_detail_teams['last activity date']
user_teams.head()

/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,user principal name,last activity date teams
0,uma_ak.i@bi.go.id,2022-04-05
1,sriyulia@bi.go.id,2022-04-05
2,andree@bi.go.id,2022-01-26
3,dwi_as@bi.go.id,2022-04-05
4,indra_m@bi.go.id,2022-04-04


# Merge & Prep

## Merge

In [257]:
user_activity_detail_onedrive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5519 entries, 0 to 5518
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   report refresh date           5519 non-null   object 
 1   user principal name           5519 non-null   object 
 2   is deleted                    5519 non-null   bool   
 3   deleted date                  0 non-null      float64
 4   last activity date            5433 non-null   object 
 5   viewed or edited file count   5519 non-null   int64  
 6   synced file count             5519 non-null   int64  
 7   shared internally file count  5519 non-null   int64  
 8   shared externally file count  5519 non-null   int64  
 9   assigned products             5515 non-null   object 
 10  report period                 5519 non-null   int64  
dtypes: bool(1), float64(1), int64(5), object(4)
memory usage: 436.7+ KB


In [267]:
user_activity_detail_teams.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6083 entries, 0 to 6082
Data columns (total 31 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   report refresh date                           6083 non-null   object 
 1   user id                                       6083 non-null   object 
 2   user principal name                           6083 non-null   object 
 3   last activity date                            6078 non-null   object 
 4   is deleted                                    6083 non-null   bool   
 5   deleted date                                  0 non-null      float64
 6   assigned products                             5623 non-null   object 
 7   team chat message count                       6083 non-null   int64  
 8   private chat message count                    6083 non-null   int64  
 9   call count                                    6083 non-null   i

In [168]:
data = master_user.merge(user_onedrive, on='user principal name', how='left')

data = data.merge(user_sharepoint, on='user principal name', how='left')
data.head()

,alternate email addresses,block credential,city,country/region,department,dirsyncenabled,display name,fax,first name,last dirsync time,last name,last password change time stamp,license assignment details,licenses,mobile phone,oath token meta data,object id,office,password never expires,phone number,postal code,preferred data location,preferred language,proxy addresses,release track,soft deletion time stamp,state,street address,strong password required,title,usage location,user principal name,when created,last activity date onedrive,report refresh date,last activity date sharepoint
0,NaN,False,JR.KANTOR PERWAKILAN BI JEMBER,NaN,JR.KANTOR PERWAKILAN BI JEMBER,True,A Robith Mubarok,NaN,A,2022-03-27 23:55:49Z,ROBITH MUBAROK,2022-03-27 23:38:01Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,479496fa-3a1b-445c-8ab5-776137c74151,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,NaN,NaN,NaN,NaN,NaN,smtp:ar_mubarok@bankindonesiagov.mail.onmicros...,NaN,NaN,KPWDN,JR.KANTOR PERWAKILAN BI JEMBER \ - \ -,NaN,Pelaksana,ID,ar_mubarok@bi.go.id,2019-07-18 07:57:32Z,2021-07-13,2022-04-05,2022-03-20
1,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,True,A. Amirullah Amir,NaN,A.,2022-03-27 23:25:44Z,AMIRULLAH AMIR,2022-03-27 23:20:10Z,NaN,Office 365 E3+Microsoft Power Automate Free+En...,NaN,NaN,0c7cb274-878f-46b6-9f19-4edb10e0df14,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,NaN,3017519,NaN,NaN,NaN,smtp:amirullah@bankindonesiagov.mail.onmicroso...,NaN,NaN,KP,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH \ -...,NaN,Manajer,ID,amirullah@bi.go.id,2019-07-18 07:55:50Z,2022-03-24,2022-04-05,2022-03-22
2,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,True,A. Donanto H. W.,NaN,ALOYSIUS,2022-03-25 10:54:17Z,DONANTO H.W.,2022-03-22 23:41:21Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,059936c1-d6d9-433b-a959-cee77e73b530,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,NaN,3018840,NaN,NaN,NaN,smtp:donanto@bankindonesiagov.mail.onmicrosoft...,NaN,NaN,KP,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYA...,NaN,Direktur,ID,donanto@bi.go.id,2019-07-18 07:55:29Z,2022-04-05,2022-04-05,2022-04-05
3,NaN,False,Kota Adm. Jakarta Pusat - Kantor Pusat,NaN,BINS.INSTITUT BI,True,A. Farid Aulia,NaN,A.,2022-03-30 05:57:42Z,FARID AULIA,2022-03-30 05:36:21Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,ea16f839-58d8-4a10-b4dd-7ab017acb051,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,NaN,3017812,NaN,NaN,NaN,smtp:arasta@bankindonesiagov.mail.onmicrosoft....,NaN,NaN,KP,BINS.INSTITUT BI \ BINS.GRUP PENGEMBANGAN AKAD...,NaN,Deputi Direktur,ID,arasta@bi.go.id,2019-07-18 07:54:50Z,2021-10-27,2022-04-05,2022-03-29
4,NaN,False,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,NaN,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,True,A. Fauzi Batubara,NaN,ACHMAD,2022-03-23 00:52:45Z,FAUZI BATUBARA,2022-03-23 00:30:35Z,NaN,Office 365 E3+Enterprise Mobility + Security E3,NaN,NaN,a08e63dd-c294-4692-8ea0-dce4db4ffc3f,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,NaN,3278221,NaN,NaN,NaN,smtp:afauzi_b@bankindonesiagov.mail.onmicrosof...,NaN,NaN,KPWDN,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTA...,NaN,Manajer,ID,afauzi_b@bi.go.id,2019-07-18 07:59:29Z,2022-03-08,2022-04-05,2022-03-08


In [164]:
data.isnull().sum()

alternate email addresses          7659
block credential                      0
city                               1494
country/region                     7466
department                         1866
dirsyncenabled                      288
display name                          0
fax                                7645
first name                          164
last dirsync time                   288
last name                           339
last password change time stamp       0
license assignment details         7659
licenses                           1805
mobile phone                       7483
oath token meta data               7659
object id                             0
office                              991
password never expires             7567
phone number                       3273
postal code                        7599
preferred data location            7659
preferred language                 7655
proxy addresses                     160
release track                      7659


In [39]:
print(len(master_user))
print(len(user_activity_detail_onedrive))
print(len(data)) # inner join
# inner join 5514 row
# left join 7659 row

7659
5519
5514


In [169]:
# report date = 05/04/2022. 180 day
# df = data[['user principal name','title','office','department','state','usage location','last activity date','viewed or edited file count','synced file count','shared internally file count','shared externally file count','report refresh date']]
df = data[['user principal name','title','office','department','state','usage location','last activity date onedrive','last activity date sharepoint', 'report refresh date']]

df.head()

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date
0,ar_mubarok@bi.go.id,Pelaksana,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,JR.KANTOR PERWAKILAN BI JEMBER,KPWDN,ID,2021-07-13,2022-03-20,2022-04-05
1,amirullah@bi.go.id,Manajer,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KP,ID,2022-03-24,2022-03-22,2022-04-05
2,donanto@bi.go.id,Direktur,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KP,ID,2022-04-05,2022-04-05,2022-04-05
3,arasta@bi.go.id,Deputi Direktur,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,BINS.INSTITUT BI,KP,ID,2021-10-27,2022-03-29,2022-04-05
4,afauzi_b@bi.go.id,Manajer,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,KPWDN,ID,2022-03-08,2022-03-08,2022-04-05


In [170]:
# df['last activity date'] = pd.to_datetime(data['last activity date'])
# df['report refresh date'] = pd.to_datetime(data['report refresh date'])

df['last activity date onedrive'] = pd.to_datetime(data['last activity date onedrive'])
df['last activity date sharepoint'] = pd.to_datetime(data['last activity date sharepoint'])
df['report refresh date'] = pd.to_datetime(data['report refresh date'])


# df['year'] = pd.DatetimeIndex(df['last activity date']).year
# df['month'] = pd.DatetimeIndex(df['last activity date']).month
# df['day'] = pd.DatetimeIndex(df['last activity date']).day

/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:6: Set

In [171]:
df['inactive days onedrive'] = df['report refresh date']-df['last activity date onedrive']
df['inactive days onedrive'] = (df['inactive days onedrive']).dt.days

df['inactive days sharepoint'] = df['report refresh date']-df['last activity date sharepoint']
df['inactive days sharepoint'] = (df['inactive days sharepoint']).dt.days

/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py

In [179]:
df.head()

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint
0,ar_mubarok@bi.go.id,Pelaksana,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,JR.KANTOR PERWAKILAN BI JEMBER,KPWDN,ID,2021-07-13,2022-03-20,2022-04-05,266.0,16.0
1,amirullah@bi.go.id,Manajer,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KP,ID,2022-03-24,2022-03-22,2022-04-05,12.0,14.0
2,donanto@bi.go.id,Direktur,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0
3,arasta@bi.go.id,Deputi Direktur,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,BINS.INSTITUT BI,KP,ID,2021-10-27,2022-03-29,2022-04-05,160.0,7.0
4,afauzi_b@bi.go.id,Manajer,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,KPWDN,ID,2022-03-08,2022-03-08,2022-04-05,28.0,28.0


In [192]:
df['active last week'] = np.where((df['inactive days onedrive']<10) | (df['inactive days sharepoint']<10), 1,0)
df['active last week onedrive'] = np.where(df['inactive days onedrive']<10, 1,0)
df['active last week sharepoint'] = np.where(df['inactive days sharepoint']<10, 1,0)





/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/datascience/conda/dataexplorationandmanipulationforcpupython3_7vy/lib/python3.7/site-packages/ipykernel_launcher.py

In [193]:
df.head()

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
0,ar_mubarok@bi.go.id,Pelaksana,ASISTEN PENYELIA PERKASAN-JR.TISM.SEKSI PENGOL...,JR.KANTOR PERWAKILAN BI JEMBER,KPWDN,ID,2021-07-13,2022-03-20,2022-04-05,266.0,16.0,0,0,0
1,amirullah@bi.go.id,Manajer,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KP,ID,2022-03-24,2022-03-22,2022-04-05,12.0,14.0,0,0,0
2,donanto@bi.go.id,Direktur,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
3,arasta@bi.go.id,Deputi Direktur,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,BINS.INSTITUT BI,KP,ID,2021-10-27,2022-03-29,2022-04-05,160.0,7.0,1,0,1
4,afauzi_b@bi.go.id,Manajer,KEPALA UNIT PUR-MDN.SPMI.UNIT LAYANAN & PENGOL...,MDN.KANTOR PERWAKILAN BI PROVINSI SUMATERA UTARA,KPWDN,ID,2022-03-08,2022-03-08,2022-04-05,28.0,28.0,0,0,0


In [220]:
# remove wrong data

df = df.loc[(df['state']=='KP')|(df['state']=='KPWDN')|(df['state']=='KPWLN')]

df = df.loc[df['department']!="'-"]

# EDA

In [221]:
# avg inactive days
a = df.groupby(by=["state"]).mean()
a

,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
state,,,,,
KP,20.095521,23.110800,0.725269,0.683659,0.540094
KPWDN,46.649417,33.089782,0.473663,0.393057,0.353951
KPWLN,15.692308,13.769231,0.821429,0.714286,0.571429


In [222]:
# percentage active user in last 1 wk
b = df.groupby(by=["department"])['active last week'].mean().sort_values(ascending=False)
# b = df.groupby(by=["department"]).mean()

b

department
BJ.KANTOR PERWAKILAN BI BEIJING                                 1.000000
DPPK.DEPARTEMEN PENGEMBANGAN PASAR KEUANGAN                     1.000000
DPD.DEPARTEMEN PENGELOLAAN DEVISA                               0.975000
UKTSI.UNIT KHUSUS TRANSFORMASI SISTEM INFORMASI                 0.972973
DHK.DEPARTEMEN HUKUM                                            0.947368
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER                     0.943182
DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN                 0.937931
DKEU.DEPARTEMEN KEUANGAN                                        0.932692
DUPK.DEPARTEMEN PENGEMBANGAN UMKM & PERLINDUNGAN KONSUMEN       0.921569
DPM.DEPARTEMEN PENGELOLAAN MONETER                              0.920000
UKPS.UNIT KHUSUS PEMBANGUNAN SPU DC & BRS                       0.918919
DKSP.DEPARTEMEN KEBIJAKAN SISTEM PEMBAYARAN                     0.910891
DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI                    0.910853
DSSK.DEPARTEMEN SURVEILANS SISTEM KEUANG

In [229]:
# percentage active user in last 1 wk
b = df.loc[df['state']=='KP'].groupby(by=["department"])['active last week'].mean().sort_values(ascending=False)
# b = df.groupby(by=["department"]).mean()

b

department
DPPK.DEPARTEMEN PENGEMBANGAN PASAR KEUANGAN                     1.000000
DPD.DEPARTEMEN PENGELOLAAN DEVISA                               0.975000
UKTSI.UNIT KHUSUS TRANSFORMASI SISTEM INFORMASI                 0.972973
DHK.DEPARTEMEN HUKUM                                            0.947368
DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER                     0.943182
DPKL.DEPARTEMEN PENGELOLAAN & KEPATUHAN LAPORAN                 0.937931
DKEU.DEPARTEMEN KEUANGAN                                        0.932692
DUPK.DEPARTEMEN PENGEMBANGAN UMKM & PERLINDUNGAN KONSUMEN       0.921569
DPM.DEPARTEMEN PENGELOLAAN MONETER                              0.920000
UKPS.UNIT KHUSUS PEMBANGUNAN SPU DC & BRS                       0.918919
DKSP.DEPARTEMEN KEBIJAKAN SISTEM PEMBAYARAN                     0.910891
DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI                    0.910853
DSSK.DEPARTEMEN SURVEILANS SISTEM KEUANGAN                      0.904762
DSTA.DEPARTEMEN STATISTIK               

In [238]:
# percentage active user in last 1 wk
# b = df.loc[df['state']=='KPWDN'].groupby(by=["department"])['active last week'].mean().sort_values(ascending=False)
b = df.loc[df['state']=='KPWDN'].groupby(by=["department"]).mean()

b

,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
department,,,,,
AB.KANTOR PERWAKILAN BI PROVINSI MALUKU,28.142857,18.777778,0.600000,0.511111,0.533333
BD.KANTOR PERWAKILAN BI PROVINSI JAWA BARAT,67.238806,32.119403,0.418367,0.316327,0.336735
BDL.KANTOR PERWAKILAN BI PROVINSI LAMPUNG,48.441860,43.255814,0.265625,0.234375,0.187500
BJM.KANTOR PERWAKILAN BI PROVINSI KALIMANTAN SELATAN,38.660714,19.946429,0.535211,0.478873,0.394366
BN.KANTOR PERWAKILAN BI PROVINSI BENGKULU,58.655172,40.677419,0.355556,0.244444,0.311111
BNA.KANTOR PERWAKILAN BI PROVINSI ACEH,43.484848,20.151515,0.545455,0.431818,0.454545
BPP.KANTOR PERWAKILAN BI BALIKPAPAN,26.192308,20.607143,0.414634,0.341463,0.317073
BTM.KANTOR PERWAKILAN BI PROVINSI KEPULAUAN RIAU,36.921569,27.392157,0.647059,0.588235,0.490196
CN.KANTOR PERWAKILAN BI CIREBON,56.568182,37.297872,0.500000,0.395833,0.437500


In [226]:
df.loc[df['state']=='KP']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
1,amirullah@bi.go.id,Manajer,KEPALA UNIT.-DEKS.UNIT MANAJEMEN INTERN,DEKS.DEPARTEMEN EKONOMI & KEUANGAN SYARIAH,KP,ID,2022-03-24,2022-03-22,2022-04-05,12.0,14.0,0,0,0
2,donanto@bi.go.id,Direktur,ANALIS EKSEKUTIF-DPSP.GESK.KEL INTEROPERABILIT...,DPSP.DEPARTEMEN PENYELENGGARAAN SISTEM PEMBAYARAN,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
3,arasta@bi.go.id,Deputi Direktur,DEKAN-BINS.AKADEMI LEADERSHIP CORPORATE ENABLE...,BINS.INSTITUT BI,KP,ID,2021-10-27,2022-03-29,2022-04-05,160.0,7.0,1,0,1
5,a_herprasetyo@bi.go.id,Staf,ASISTEN PENYELIA-DSTA.DQAD.FUNGSI DISEMINASI S...,DSTA.DEPARTEMEN STATISTIK,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
6,aindris@bi.go.id,Asisten Manajer,KEPALA SEKSI.-DKEU.UMI.SEKSI KEUANGAN,DKEU.DEPARTEMEN KEUANGAN,KP,ID,2022-03-29,2022-03-18,2022-04-05,7.0,18.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7649,zulfia_f@bi.go.id,Asisten Direktur,EKONOM SENIOR-DKEM.GEM.KEL MONETER & SEKTOR KE...,DKEM.DEPARTEMEN KEBIJAKAN EKONOMI & MONETER,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
7651,zulham@bi.go.id,Manajer,KEPALA UNIT-DPPT.PTT.UNIT PENYELESAIAN TRANSAK...,DPPT.DEPARTEMEN JASA PERBANKAN PERIZINAN&OPERA...,KP,ID,2022-03-25,2022-03-25,2022-04-05,11.0,11.0,0,0,0
7653,zulkarnaen@bi.go.id,Staf,ASISTEN PENYELIA-DPPT.PTT.UNIT PENYELESAIAN TR...,DPPT.DEPARTEMEN JASA PERBANKAN PERIZINAN&OPERA...,KP,ID,2022-04-05,2022-03-30,2022-04-05,0.0,6.0,1,1,1
7656,zulmi_r@bi.go.id,Manajer,ANALIS-DSDM.GKSM.KEL KEBIJAKAN MANAJEMEN BUDAY...,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1


In [128]:
# with pd.ExcelWriter('inactive-days.xlsx') as writer:  
#     a.to_excel(writer, sheet_name='kp vs kpw vs kpwln')
#     b.to_excel(writer, sheet_name='satker')

# a.to_csv('inactive-days-state.csv')
# b.to_csv('inactive-days-satker.csv')

In [241]:
df[df['department']=='DPU.DEPARTEMEN PENGELOLAAN UANG']
# df[df['department']=='DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
7,a_lukman@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-DPU.DLK.SEKSI MSUK,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-03-28,2022-03-28,2022-04-05,8.0,8.0,1,1,1
25,abdul_bi@bi.go.id,Pelaksana,ADMINISTRATOR-DPU.GP2U.KEL DATA & INFORMASI ED...,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-03-28,2022-04-01,2022-04-05,8.0,4.0,1,1,1
28,abdul_f@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-DPU.DLK.SEKSI SHUM,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-03-10,2022-03-30,2022-04-05,26.0,6.0,1,0,1
38,abdul_majid@bi.go.id,Asisten Manajer,ANALIS YUNIOR-DPU.GKPU.KEL PERENCANAAN OPERASI...,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
42,abdul_s@bi.go.id,Staf,ASISTEN PENYELIA PERKASAN-DPU.DLK.SEKSI SHUM,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-04-05,2022-03-31,2022-04-05,0.0,5.0,1,1,1
49,abdur_rz@bi.go.id,Staf,ASISTEN PENYELIA PERKASAN-DPU.DLK.SEKSI LAYANA...,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-03-22,2022-02-23,2022-04-05,14.0,41.0,0,0,0
86,jainuri@bi.go.id,Asisten Direktur,KEPALA TIM-DPU.DDUN.TIM DISTRIBUSI WILAYAH TIMUR,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2021-01-21,2021-10-04,2022-04-05,439.0,183.0,0,0,0
96,achmad_sulaeman@bi.go.id,Asisten,ASISTEN ADMINISTRATOR PERKASAN-DPU.DLK.SEKSI L...,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-01-17,2022-02-17,2022-04-05,78.0,47.0,0,0,0
111,ade_ashadi@bi.go.id,Asisten Manajer,KEPALA SEKSI PUR-DPU.DDUN.SEKSI PENGELOLAAN KH...,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-04-04,2022-01-12,2022-04-05,1.0,83.0,1,1,0
114,ade_im@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-DPU.DLK.SEKSI SHUM,DPU.DEPARTEMEN PENGELOLAAN UANG,KP,ID,2022-02-09,2022-03-07,2022-04-05,55.0,29.0,0,0,0


In [242]:
df[df['department']=='DSDM.DEPARTEMEN SUMBER DAYA MANUSIA']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
10,pandu@bi.go.id,Asisten Direktur,ANALIS SENIOR-DSDM.GPSM.KEL MITRA BISNIS SDM &...,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
47,abdullah_m@bi.go.id,Asisten Manajer,ASISTEN PENYELIA-DSDM.PKMI.SEKSI KESELAMATAN &...,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-04,2022-03-23,2022-04-05,1.0,13.0,1,1,0
50,abdurahman_zaki.pcpm@bi.go.id,Asisten Manajer,CALON PEGAWAI-DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,US,NaT,NaT,NaT,NaN,NaN,0,0,0
81,achmad_fahmi@bi.go.id,Pelaksana,ADMINISTRATOR.-DSDM.PKMI.UNIT KEUANGAN,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-05,2022-04-04,2022-04-05,0.0,1.0,1,1,1
100,achmad_zacky.pcpm@bi.go.id,Asisten Manajer,CALON PEGAWAI-DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,NaT,NaT,NaT,NaN,NaN,0,0,0
124,adesum@bi.go.id,Manajer,KEPALA UNIT-DSDM.RKS.UNIT PENGELOLAAN KONTRAKTUAL,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0,1,1,1
140,adhira_azhari@bi.go.id,Asisten Manajer,ANALIS YUNIOR-DSDM.GKSM.KEL KEBIJAKAN IMPLEMEN...,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2022-04-05,2022-04-04,2022-04-05,0.0,1.0,1,1,1
145,adhy_rp@bi.go.id,Asisten Manajer,PEGAWAI PTB-DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2021-10-11,2021-10-11,2022-04-05,176.0,176.0,0,0,0
165,adidoyo@bi.go.id,Deputi Direktur,PEGAWAI PENUGASAN-DSDM.DEPARTEMEN SUMBER DAYA ...,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,ID,2021-11-22,2021-11-29,2022-04-05,134.0,127.0,0,0,0
171,adinda_shafira.pcpm@bi.go.id,Asisten Manajer,CALON PEGAWAI-DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,DSDM.DEPARTEMEN SUMBER DAYA MANUSIA,KP,NaN,NaT,NaT,NaT,NaN,NaN,0,0,0


In [174]:
df[df['department']=='YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIMEWA YOGYAKARTA']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint
179,aditya_bp@bi.go.id,Asisten,ASISTEN ADMINISTRATOR PERKASAN-YK.SPMI.SEKSI P...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,NaN,NaT,NaT,NaT,NaN,NaN
387,ahmad_yi@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-YK.SPMI.SEKSI PENGOLAHA...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,2021-11-25,2022-04-05,2022-04-05,131.0,0.0
624,andipalupi@bi.go.id,Asisten Direktur,KEPALA TIM-YK.TIM IMPLEMENTASI KEKDA,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,2022-04-05,2022-04-01,2022-04-05,0.0,4.0
705,andriyanto1@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-YK.SPMI.SEKSI LAYANAN B...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,2020-11-26,2022-02-01,2022-04-05,495.0,63.0
707,andriyono_r@bi.go.id,Staf,ASISTEN ANALIS-YK.TIK.FUNGSI PELAKSANAAN PENGE...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,2022-02-21,2022-04-01,2022-04-05,43.0,4.0
746,anggraeni_ad@bi.go.id,Pelaksana,ADMINISTRATOR.-YK.SPUR.TIM MANAJEMEN INTERN,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,US,NaT,NaT,NaT,NaN,NaN
1078,atnan_p@bi.go.id,Pelaksana,ASISTEN PENYELIA.-YK.SPUR.UNIT KEUANGAN,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,US,NaT,NaT,NaT,NaN,NaN
1143,azman_yattaqi@bi.go.id,Pelaksana,ADMINISTRATOR PERKASAN-YK.SPMI.SEKSI PENGOLAHA...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,NaN,NaT,NaT,NaT,NaN,NaN
1204,banu_t@bi.go.id,Asisten Manajer,ANALIS YUNIOR-YK.TIK.FUNGSI PELAKSANAAN PENGEM...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,2021-07-16,2021-12-28,2022-04-05,263.0,98.0
1236,beni_s@bi.go.id,Asisten,ASISTEN PENGAWAS PAM-YK.SPUR.FUNGSI LOGISTIK S...,YK.KANTOR PERWAKILAN BI PROVINSI DAERAH ISTIME...,KPWDN,ID,NaT,NaT,NaT,NaN,NaN


In [132]:
df[df['state']=='Sumatera Utara']

,user principal name,title,office,department,state,usage location,last activity date,viewed or edited file count,synced file count,shared internally file count,shared externally file count,report refresh date,inactive days
4242,riadi.i@bi.go.id,NaN,THI - KPw Sumut,NaN,Sumatera Utara,ID,2022-04-05,130,0,7,0,2022-04-05,0.0


In [175]:
df[df['user principal name']=='kurniawan_eka@bi.go.id']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint
4084,kurniawan_eka@bi.go.id,Asisten Manajer,ANALIS TI YUNIOR-DPSI.DAE.FUNGSI ARSITEKTUR TE...,DPSI.DEPARTEMEN PENGELOLAAN SISTEM INFORMASI,KP,ID,2022-04-05,2022-04-05,2022-04-05,0.0,0.0


In [245]:
df[df['user principal name']=='aries_p@bi.go.id']

,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint
934,aries_p@bi.go.id,Asisten,ASISTEN PENGAWAS PAM-JR.SPUR.FUNGSI LOGISTIK S...,JR.KANTOR PERWAKILAN BI JEMBER,KPWDN,NaN,NaT,NaT,NaT,NaN,NaN,0,0,0


,user principal name,title,office,department,state,usage location,last activity date onedrive,last activity date sharepoint,report refresh date,inactive days onedrive,inactive days sharepoint,active last week,active last week onedrive,active last week sharepoint


In [141]:
len(df[df['inactive days']<30])

4386

In [142]:
len(df)

5514

In [145]:
len(master_user)

7659

In [146]:
4386/7659

0.572659616137877

In [ ]:
!conda install -c conda-forge jupyter_contrib_nbextensions -y
